# Market Basket Analysis with SQL

# Project Overview

Whether you shop from meticulously planned grocery lists or let whimsy guide your grazing, our unique food rituals define who we are. Instacart, a grocery ordering and delivery app, aims to make it easy to fill your refrigerator and pantry with your personal favorites and staples when you need them.

In this project, I will use this anonymized data from Instacart on customer orders over time to predict which previously purchased products will be in a user’s next order.

## What I Have Learned From This Project

* SQLAlchemy<br>
* SQL queries: CREATE, SELECT, FROM, JOIN, DROP, INNER JOIN, etc.<br>
* Using PostgreSQL database<br>
* Association rule<br>

# Data Description

The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order. The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order. We also provide the week and hour of day the order was placed, and a relative measure of time between orders. Each entity (customer, product, order, aisle, etc.) has an associated unique id. Most of the files and variable names should be self-explanatory.

More information about this dataset can be found [here](https://www.kaggle.com/c/instacart-market-basket-analysis/data).

# Load the Data into PostgreSQL Database

In [1]:
# import necessary tools 
from sqlalchemy import create_engine
import sql
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

pd.options.mode.chained_assignment = None

In [2]:
import os
print(os.getcwd())

/Users/andreduong/market-basket-analysis


In [3]:
data_path = '/Users/andreduong/market-basket-analysis'
data_output = '/Users/andreduong/market-basket-analysis'

In [4]:
from subprocess import check_output

conn = create_engine('postgresql:///instacart')

In [5]:
# read chunk to postgres
# this python script is influenced by https://gist.github.com/olgabradford/f04f23692c78fc0beb377894ce5e5e59
def read_chunk_csv_to_psql(file_name, disk_engine, table):
    # to time loading process
    start = datetime.now()
    # chunk_size - number of rows
    chunk_size = 10000
    i = 0
    index_start = 1
    drop_table = "DROP TABLE IF EXISTS %s ;" %(table)
    conn.execute(drop_table)        
    
    for df in pd.read_csv(file_name, chunksize=chunk_size, iterator=True, encoding='utf-8'):
        df = df.rename(columns = {c: c.replace(' ', '') for c in df.columns}) # delete spaces between columns from CSV file
        df.index += index_start
        i += 1
        if i <= 50:
            df.to_sql(table, disk_engine, if_exists = 'append')
            if i%10 == 0:
                #print how long it takes to load database
                print ('{} Seconds: Loaded rows {}'.format((datetime.now() - start).total_seconds(), i*chunk_size))
            index_start = df.index[-1] + 1
        else:
            print("Limit Data for exploration")
            break
    
    # created indexes on all id columns
    index_columns = [col for col in df.columns if col.find("_id") > -1]
    for col in index_columns:
        # create indexes for latter for quicker access
        create_indexes = "CREATE INDEX index_%s on %s (%s);" %(col+ "_" + table, table, col)
        conn.execute(create_indexes)        

In [6]:
print(os.getcwd())
files_list = [file for file in os.listdir() if file.find(".csv") != -1]
print(files_list)

/Users/andreduong/market-basket-analysis
['products.csv', 'orders.csv', 'order_products__train.csv', 'departments.csv', 'aisles.csv', 'order_products__prior.csv', 'sample_submission.csv']


In [7]:
for file in files_list:    
    table = file.split(".")[0]
    print('\nloading table {}'.format(table))
    file_name = file
    read_chunk_csv_to_psql(file_name,conn,table)
    print ('loading of table {} complete'.format(table))


loading table products
loading of table products complete

loading table orders
17.615521 Seconds: Loaded rows 100000
37.885305 Seconds: Loaded rows 200000
69.787772 Seconds: Loaded rows 300000
94.289425 Seconds: Loaded rows 400000
117.934345 Seconds: Loaded rows 500000
Limit Data for exploration
loading of table orders complete

loading table order_products__train
22.270205 Seconds: Loaded rows 100000
40.803577 Seconds: Loaded rows 200000
59.619467 Seconds: Loaded rows 300000
79.089541 Seconds: Loaded rows 400000
95.195659 Seconds: Loaded rows 500000
Limit Data for exploration
loading of table order_products__train complete

loading table departments
loading of table departments complete

loading table aisles
loading of table aisles complete

loading table order_products__prior
17.48114 Seconds: Loaded rows 100000
33.202361 Seconds: Loaded rows 200000
51.615399 Seconds: Loaded rows 300000
67.736884 Seconds: Loaded rows 400000
85.495928 Seconds: Loaded rows 500000
Limit Data for explo

# Let's take a look at each table

## aisles

In [8]:
aisles = pd.read_sql_query("SELECT * FROM aisles LIMIT 5;", conn)
print("Top 5 in aisles table:\n", aisles.head())

Top 5 in aisles table:
    index  aisle_id                       aisle
0      1         1       prepared soups salads
1      2         2           specialty cheeses
2      3         3         energy granola bars
3      4         4               instant foods
4      5         5  marinades meat preparation


## departments

In [9]:
departments = pd.read_sql_query("SELECT * FROM departments LIMIT 5;", conn)
print("Top 5 in departments table:\n", departments.head())

Top 5 in departments table:
    index  department_id department
0      1              1     frozen
1      2              2      other
2      3              3     bakery
3      4              4    produce
4      5              5    alcohol


## products

In [10]:
products = pd.read_sql_query("SELECT * FROM products LIMIT 5;", conn)
print("Top 5 in products table:\n", products.head())

Top 5 in products table:
    index  product_id                                       product_name  \
0      1           1                         Chocolate Sandwich Cookies   
1      2           2                                   All-Seasons Salt   
2      3           3               Robust Golden Unsweetened Oolong Tea   
3      4           4  Smart Ones Classic Favorites Mini Rigatoni Wit...   
4      5           5                          Green Chile Anytime Sauce   

   aisle_id  department_id  
0        61             19  
1       104             13  
2        94              7  
3        38              1  
4         5             13  


## orders

In [11]:
orders = pd.read_sql_query("SELECT * FROM orders LIMIT 5;", conn)
print("Top 5 in orders table:\n", orders.head())

Top 5 in orders table:
    index  order_id  user_id eval_set  order_number  order_dow  \
0      1   2539329        1    prior             1          2   
1      2   2398795        1    prior             2          3   
2      3    473747        1    prior             3          3   
3      4   2254736        1    prior             4          4   
4      5    431534        1    prior             5          4   

   order_hour_of_day  days_since_prior_order  
0                  8                     NaN  
1                  7                    15.0  
2                 12                    21.0  
3                  7                    29.0  
4                 15                    28.0  


## order_products__train

In [12]:
orders_train = pd.read_sql_query("SELECT * FROM order_products__train LIMIT 5;", conn)
print("Top 5 in orders train table:\n", orders_train.head())

Top 5 in orders train table:
    index  order_id  product_id  add_to_cart_order  reordered
0      1         1       49302                  1          1
1      2         1       11109                  2          1
2      3         1       10246                  3          0
3      4         1       49683                  4          0
4      5         1       43633                  5          1


## order_products__prior

In [13]:
orders_prior = pd.read_sql_query("SELECT * FROM order_products__prior LIMIT 5;", conn)
print("Top 5 in orders prior table:\n", orders_prior.head())

Top 5 in orders prior table:
    index  order_id  product_id  add_to_cart_order  reordered
0      1         2       33120                  1          1
1      2         2       28985                  2          1
2      3         2        9327                  3          0
3      4         2       45918                  4          1
4      5         2       30035                  5          0


In [14]:
print("All loaded successfully. Now we can delete temporary dataframes.")

All loaded successfully. Now we can delete temporary dataframes.


In [15]:
del aisles, departments, products, orders, orders_train, orders_prior

# Merging tables

## department, aisle and products

In [18]:
# merge tables department, aisle, products, and call it productscombined
# add a time counter
start_time1 = datetime.now()

drop_productscombined_table = """ DROP TABLE IF EXISTS productscombined;"""
conn.execute(drop_productscombined_table)
joinprod_sql = """\
    CREATE TABLE productscombined AS
    SELECT p.*, d.department, a.aisle
    FROM products p
    INNER JOIN departments d ON p.department_id = d.department_id
    INNER JOIN aisles a ON p.aisle_id = a.aisle_id;
    """
conn.execute(joinprod_sql)

# load first 5 rows
productscombined_qc = pd.read_sql_query("SELECT * FROM productscombined LIMIT 5;", conn)

end_time1 = datetime.now()

print('Time for productscombined: {}\n'.format((end_time1 - start_time1).total_seconds()))
print("Top 5 in productscombined:\n", productscombined_qc.head())

count_merge1 = pd.read_sql_query("SELECT COUNT(product_id) FROM productscombined;", conn)
print("\nTotal number of products in database: \n", count_merge1)

Time for productscombined: 0.1987

Top 5 in productscombined:
    index  product_id                                       product_name  \
0      1           1                         Chocolate Sandwich Cookies   
1      2           2                                   All-Seasons Salt   
2      3           3               Robust Golden Unsweetened Oolong Tea   
3      4           4  Smart Ones Classic Favorites Mini Rigatoni Wit...   
4      5           5                          Green Chile Anytime Sauce   

   aisle_id  department_id department                       aisle  
0        61             19     snacks               cookies cakes  
1       104             13     pantry           spices seasonings  
2        94              7  beverages                         tea  
3        38              1     frozen                frozen meals  
4         5             13     pantry  marinades meat preparation  

Total number of products in database: 
    count
0  49688


## order_products_prior and orders

In [23]:
start_time2 = datetime.now()

drop_orderscombined1_table=""" DROP TABLE IF EXISTS orderscombined1;"""
conn.execute(drop_orderscombined1_table)
joinordersprior_sql1 = """\
    CREATE TABLE orderscombined1 AS
    SELECT o.*, op.product_id, op.add_to_cart_order,op.reordered
    FROM orders o
    INNER JOIN order_products__prior op ON o.order_id = op.order_id;
    """
conn.execute(joinordersprior_sql1)

# load top 5
orderscombined1 = pd.read_sql_query("SELECT * FROM orderscombined1 LIMIT 5;", conn)

# create indexes to do requests much faster
indexcolumns = [col for col in orderscombined1.columns if col.find("_id")>-1]
for col in indexcolumns:
    createindexes = "CREATE INDEX index_%s ON orderscombined1 (%s);" % (col + "_orderscombined1", col)
    conn.execute(createindexes)         
        
end_time2 = datetime.now()
print('\nTime for orderscombined1: {}\n'.format((end_time2-start_time2).total_seconds()))
print("Top 5 orderscombined1 table: \n", orderscombined1.head())

count_merge1 = pd.read_sql_query("SELECT COUNT(product_id) FROM orderscombined1;", conn)
print("\nTotal number of products in orders in database \n", count_merge1)


Time for orderscombined1: 0.822446

Top 5 orderscombined1 table: 
      index  order_id  user_id eval_set  order_number  order_dow  \
0  7400621         6    22352    prior             4          1   
1  7400621         6    22352    prior             4          1   
2  7400621         6    22352    prior             4          1   
3   200029         8     3107    prior             5          4   
4  4951076        14    18194    prior            49          3   

   order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                 12                    30.0       40462                  1   
1                 12                    30.0       15873                  2   
2                 12                    30.0       41897                  3   
3                  6                    17.0       23423                  1   
4                 15                     3.0       20392                  1   

   reordered  
0          0  
1          0  
2        

## orders and the order_products__train

In [24]:
start_time3 = datetime.now()

drop_orderscombined2_table=""" DROP TABLE IF EXISTS orderscombined2;"""
conn.execute(drop_orderscombined2_table)
joinordersprior_sql2 = """\
    CREATE TABLE orderscombined2 AS
    SELECT o.*, op.product_id, op.add_to_cart_order,op.reordered
    FROM orders o
    INNER JOIN order_products__train op ON o.order_id = op.order_id;
    """

conn.execute(joinordersprior_sql2)
# load top 5
orderscombined2 = pd.read_sql_query("SELECT * FROM orderscombined2 LIMIT 5;", conn)

#create indexes to do requests much faster
indexcolumns = [col for col in orderscombined2.columns if col.find("_id") > -1]
for col in indexcolumns:
    createindexes = "CREATE INDEX index_%s ON orderscombined2 (%s);" %(col + "_orderscombined2", col)
    conn.execute(createindexes)              
        
end_time3 = datetime.now()
print('\nTime for orderscombined2: {}\n'.format((end_time3-start_time3).total_seconds()))
print("Top 5 orderscombined2 table: \n", orderscombined2.head())

count_merge2 = pd.read_sql_query("SELECT COUNT (user_id) FROM orderscombined2 ;", conn)
print("\nTotal number of products in orders in database \n", count_merge2)


Time for orderscombined2: 1.187384

Top 5 orderscombined2 table: 
      index  order_id  user_id eval_set  order_number  order_dow  \
0  4344949        96    17227    train             7          6   
1  4344949        96    17227    train             7          6   
2  4344949        96    17227    train             7          6   
3  4344949        96    17227    train             7          6   
4  4344949        96    17227    train             7          6   

   order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                 20                    30.0       20574                  1   
1                 20                    30.0       30391                  2   
2                 20                    30.0       40706                  3   
3                 20                    30.0       25610                  4   
4                 20                    30.0       27966                  5   

   reordered  
0          1  
1          0  
2        

## Merge all tables

In [30]:
start_time3 = datetime.now()

drop_prior_merge=""" DROP TABLE IF EXISTS prioralldata;"""
conn.execute(drop_prior_merge)
join_prioralldata_sql = """\
    CREATE TABLE prioralldata AS
    SELECT o.*, gd.product_name, gd.department, gd.aisle
    FROM orderscombined1 o
    INNER JOIN productscombined gd
    ON o.product_id = gd.product_id;
    """
conn.execute(join_prioralldata_sql)
priorall_qc = pd.read_sql_query("SELECT * FROM prioralldata LIMIT 5;", conn)

# create indexes
indexcolumns = [col for col in orderscombined1.columns if col.find("_id") > -1]
for col in indexcolumns:
    createindexes = "CREATE INDEX index_%s on prioralldata (%s);" %(col + "_prioralldata", col)
    conn.execute(createindexes)
        
end_time3 = datetime.now()
print('\nTime for prioralldata table: {}\n'.format((end_time3-start_time3).total_seconds()))
print("Prioralldata table: \n", priorall_qc.head())
count_prior = pd.read_sql_query("SELECT COUNT (order_id) FROM prioralldata;", conn)
print("\nCount orders prior table:\n",count_prior)

# delete all temporary QC outputs
del priorall_qc, count_prior


Time for prioralldata table: 0.978575

Prioralldata table: 
      index  order_id  user_id eval_set  order_number  order_dow  \
0   200029         8     3107    prior             5          4   
1  4951076        14    18194    prior            49          3   
2  4951076        14    18194    prior            49          3   
3  4951076        14    18194    prior            49          3   
4  4951076        14    18194    prior            49          3   

   order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                  6                    17.0       23423                  1   
1                 15                     3.0       20392                  1   
2                 15                     3.0       27845                  2   
3                 15                     3.0         162                  3   
4                 15                     3.0        2452                  4   

   reordered                    product_name  department  \


In [32]:
start_time3 = datetime.now()

drop_prior_merge=""" DROP TABLE IF EXISTS trainalldata;"""
conn.execute(drop_prior_merge)
join_trainalldata_sql = """\
    CREATE TABLE trainalldata AS
    SELECT o.*, gd.product_name, gd.department, gd.aisle
    FROM orderscombined2 o
    INNER JOIN productscombined gd
    ON o.product_id = gd.product_id;
    """
conn.execute(join_trainalldata_sql)
trainall_qc = pd.read_sql_query("SELECT * FROM trainalldata LIMIT 5;", conn)

# create indexes
indexcolumns = [col for col in orderscombined2.columns if col.find("_id") > -1]
for col in indexcolumns:
    createindexes="CREATE INDEX index_%s on trainalldata (%s);" %(col+"_trainalldata", col)
    conn.execute(createindexes)        
        
end_time3 = datetime.now()
print('\nTime for trainalldata table: {}\n'.format((end_time3-start_time3).total_seconds()))
print("Trainalldata table: \n", trainall_qc.head())

count_train = pd.read_sql_query("SELECT COUNT (order_id) FROM trainalldata ;", conn)
print("\nCount orders train table:\n",count_train)

# delete all temporary QC outputs
del trainall_qc, count_train


Time for trainalldata table: 0.899237

Trainalldata table: 
      index  order_id  user_id eval_set  order_number  order_dow  \
0  4344949        96    17227    train             7          6   
1  4344949        96    17227    train             7          6   
2  4344949        96    17227    train             7          6   
3  4344949        96    17227    train             7          6   
4  4344949        96    17227    train             7          6   

   order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                 20                    30.0       20574                  1   
1                 20                    30.0       30391                  2   
2                 20                    30.0       40706                  3   
3                 20                    30.0       25610                  4   
4                 20                    30.0       27966                  5   

   reordered                 product_name department  \
0   